In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.sprint_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
query_all_laps=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.sprint_lap_time`
UNION ALL
SELECT * FROM `perceptive-ivy-290216.f1_api.final_driver_positions_all_sprint`
"""
track_all_laps=pandas_gbq.read_gbq(query_all_laps,project_id,dialect='standard')

Downloading: 100%|██████████|


In [8]:
race="United States Grand Prix"

In [9]:
track2=track3[(track3["GP"]==race)&(track3["Year"]==2024)]

In [10]:
#Get Final Pos to order the final plot
track_all_laps_2=track_all_laps[(track_all_laps["GP"]==race)&(track_all_laps["Year"]==2024)]
final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]

In [11]:
final_pos

,Driver,Position
5842,VER,1.0
5858,SAI,2.0
5890,LEC,4.0
5907,RUS,5.0
5923,HAM,6.0
5941,MAG,7.0
5958,HUL,8.0
5974,PER,9.0
5989,PIA,10.0
6005,TSU,11.0


In [12]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]
Drivers=(track2['Driver'].unique())

In [13]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,MEDIUM,19
1,ALO,1.0,MEDIUM,19
2,BOT,1.0,MEDIUM,19
3,COL,1.0,MEDIUM,19
4,GAS,1.0,MEDIUM,19


In [14]:
stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position'])

In [15]:
fig=px.bar(
    stint_with_pos,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="simple_white",
    title="<b>Tyre Strategies for the {} {} Sprint Race</b>".format(year,gp),
    height=800, width=1200,
    hover_data=["Position"],
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="Black",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    margin=dict(l=5, r=5, t=35, b=5),
    )

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [16]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Tyre Strat/{}/{} {} Sprint Race Tyre Strategies.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Race Plot

In [8]:
#2024
# race_multi=['Austrian Grand Prix','Miami Grand Prix','Chinese Grand Prix']
#2023
# race_multi=['Austrian Grand Prix','Azerbaijan Grand Prix','Belgian Grand Prix','Qatar Grand Prix','São Paulo Grand Prix','United States Grand Prix']
#2022
# race_multi=['Austrian Grand Prix','Emilia Romagna Grand Prix','São Paulo Grand Prix']
#2021
race_multi=['British Grand Prix','Italian Grand Prix','São Paulo Grand Prix']

year_multi=2021
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    track_all_laps_2=track_all_laps[(track_all_laps["GP"]==i)&(track_all_laps["Year"]==year_multi)]
    final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    Drivers=(track2['Driver'].unique())
    stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
    stints = stints.rename(columns={"LapNumber": "StintLength"})
    stints.head()
    stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
    stint_with_pos.head()
    fig=px.bar(
        stint_with_pos,
        y="Driver",
        x='StintLength',
        text="StintLength",
        orientation='h',
        barmode='stack',
        color='Compound',
        template="simple_white",
        title="<b>Tyre Strategies for the {} {}</b>".format(year_multi,i),
        height=800, width=1200,
        hover_data=["Position"],
        color_discrete_map={
                    "SOFT": "#F91536",
                    "MEDIUM": "#ffdf00 ",
                    "HARD":"#c8c8c8 ",
                    "INTERMEDIATE":"#009E60 ",
                    "WET":"#3671C6"
        }
    )
    fig.update_layout(
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        xaxis_title="<b>Lap Number</b>",
        yaxis_title="<b>Driver</b>",
        title_x=0.5,
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )

    fig.update_layout(yaxis={'categoryorder':'total descending'},    
            hoverlabel=dict(
            bgcolor="Black",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        margin=dict(l=5, r=5, t=35, b=5),
        )

    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

    fig.update_yaxes(ticksuffix = "  ")
    fig.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Tyre Strat/{}/{} {} Sprint Race Tyre Strategies.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')
